In [22]:
import pandas as pd
import numpy as np
import random
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Semillas
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# Config
MODEL_NAME = "bert-base-uncased"
TARGET = "gender"
CSV_PATH = "features_linguisticas_es_con_glove_con_ideology.csv"
COHEN_PATH = "cohens_results.csv"
COHEN_THRESHOLD = 0.10
BATCH_SIZE = 16
EPOCHS = 10
LR = 3e-5
FOLDS = 5
EARLY_STOPPING_PATIENCE = 4
MAX_LENGTH = 128

# Datos
df = pd.read_csv(CSV_PATH)
cohen_df = pd.read_csv(COHEN_PATH)

def normalize_gender(g):
    if pd.isnull(g): return np.nan
    g = str(g).strip().lower()
    if g in ["m", "male", "masculino"]: return "male"
    elif g in ["f", "female", "femenino"]: return "female"
    return np.nan

df["gender"] = df["gender"].apply(normalize_gender)
df = df[df["gender"].isin(["male", "female"])].copy()
df["label"] = df["gender"].map({"male": 0, "female": 1})

# Variables relevantes
selected_vars = cohen_df[
    (cohen_df["target"] == TARGET) &
    (cohen_df["cohens_d"].abs() > COHEN_THRESHOLD)
]["variable"].unique().tolist()

print(f"🔎 Usando {len(selected_vars)} variables GloVe con |d| > {COHEN_THRESHOLD}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
text_col = "clean_text"
df = df[[text_col, "label"] + selected_vars].dropna().reset_index(drop=True)

# Dataset
class MultiModalDataset(Dataset):
    def __init__(self, texts, nums, labels):
        self.texts = texts
        self.nums = nums
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        enc = tokenizer(self.texts[idx], truncation=True, padding="max_length", max_length=MAX_LENGTH, return_tensors="pt")
        item = {k: v.squeeze() for k, v in enc.items()}
        item["nums"] = torch.tensor(self.nums[idx], dtype=torch.float)
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

# Modelo
class TransformerWithTabular(nn.Module):
    def __init__(self, transformer_name, num_tabular_features):
        super().__init__()
        self.transformer = AutoModel.from_pretrained(transformer_name)
        for name, param in self.transformer.named_parameters():
            if "encoder.layer.11" not in name:
                param.requires_grad = False

        self.tabular_net = nn.Sequential(
            nn.Linear(num_tabular_features, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 64)
        )
        self.classifier = nn.Sequential(
            nn.Linear(768 + 64, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 2)
        )

    def forward(self, input_ids, attention_mask, nums):
        out = self.transformer(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = out.last_hidden_state[:, 0]
        tabular_out = self.tabular_net(nums)
        combined = torch.cat([cls_output, tabular_out], dim=1)
        return self.classifier(combined)

# Preparación
texts = df[text_col].tolist()
features = df[selected_vars].values.astype(np.float32)
labels = df["label"].values

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_weights = compute_class_weight("balanced", classes=np.unique(labels), y=labels)
weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)

skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)
f1_scores = []

# Entrenamiento
for fold, (train_idx, val_idx) in enumerate(skf.split(texts, labels)):
    print(f"\n📦 Fold {fold+1}/{FOLDS}")

    X_train_texts = [texts[i] for i in train_idx]
    X_val_texts = [texts[i] for i in val_idx]
    y_train = labels[train_idx]
    y_val = labels[val_idx]
    # Oversampling manual de la clase minoritaria (female)
    X_train_feats_orig = features[train_idx]
    X_val_feats = features[val_idx]
    
    scaler = StandardScaler()
    X_train_feats_scaled = scaler.fit_transform(X_train_feats_orig)
    X_val_feats_scaled = scaler.transform(X_val_feats)
    
    X_train_texts_aug = list(X_train_texts)
    X_train_feats_aug = list(X_train_feats_scaled)
    y_train_aug = list(y_train)
    
    # Duplicar instancias de clase 'female' (label = 1)
    for i in range(len(y_train)):
        if y_train[i] == 1:
            X_train_texts_aug.append(X_train_texts[i])
            X_train_feats_aug.append(X_train_feats_scaled[i])
            y_train_aug.append(1)
    
    # Convertir a arrays
    X_train_feats_aug = np.array(X_train_feats_aug)
    y_train_aug = np.array(y_train_aug)

    scaler = StandardScaler()
    X_train_feats = scaler.fit_transform(features[train_idx])
    X_val_feats = scaler.transform(features[val_idx])

    train_ds = MultiModalDataset(X_train_texts, X_train_feats, y_train)
    val_ds = MultiModalDataset(X_val_texts, X_val_feats, y_val)
    train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE)

    model = TransformerWithTabular(MODEL_NAME, len(selected_vars)).to(device)
    opt = torch.optim.AdamW(model.parameters(), lr=LR)
    scheduler = ReduceLROnPlateau(opt, mode='max', factor=0.5, patience=2)
    loss_fn = nn.CrossEntropyLoss(weight=weights_tensor)

    best_f1 = 0
    patience = EARLY_STOPPING_PATIENCE

    for epoch in range(EPOCHS):
        model.train()
        total_loss = 0
        for batch in tqdm(train_dl, desc=f"Fold {fold+1} Epoch {epoch+1}"):
            input_ids = batch["input_ids"].to(device)
            attn = batch["attention_mask"].to(device)
            nums = batch["nums"].to(device)
            lbls = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attn, nums=nums)
            loss = loss_fn(outputs, lbls)

            opt.zero_grad()
            loss.backward()
            opt.step()
            total_loss += loss.item()
        print(f"🔁 Epoch {epoch+1} loss: {total_loss/len(train_dl):.4f}")

        model.eval()
        all_preds, all_lbls = [], []
        with torch.no_grad():
            for batch in val_dl:
                input_ids = batch["input_ids"].to(device)
                attn = batch["attention_mask"].to(device)
                nums = batch["nums"].to(device)
                lbls = batch["labels"].to(device)

                outputs = model(input_ids=input_ids, attention_mask=attn, nums=nums)
                preds = torch.argmax(outputs, dim=1).cpu().numpy()
                all_preds.extend(preds)
                all_lbls.extend(lbls.cpu().numpy())

        f1 = f1_score(all_lbls, all_preds, average="macro")
        prec = precision_score(all_lbls, all_preds, average="macro", zero_division=0)
        rec = recall_score(all_lbls, all_preds, average="macro", zero_division=0)
        print(f"✅ F1: {f1:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}")

        scheduler.step(f1)

        if f1 > best_f1:
            best_f1 = f1
            patience = EARLY_STOPPING_PATIENCE
        else:
            patience -= 1
            if patience == 0:
                print("⏹️ Early stopping")
                break

    f1_scores.append(best_f1)
    print("📊 Matriz de confusión:")
    print(confusion_matrix(all_lbls, all_preds))
    from sklearn.metrics import classification_report
    print(classification_report(all_lbls, all_preds, target_names=["male", "female"]))

# Resultados finales
f1_avg = np.mean(f1_scores)
print("\n📊 F1 macro por fold:", [round(f, 4) for f in f1_scores])
print(f"🏁 F1 macro promedio final: {f1_avg:.4f}")


🔎 Usando 5 variables GloVe con |d| > 0.1

📦 Fold 1/5


Fold 1 Epoch 1: 100%|██████████| 147/147 [00:04<00:00, 31.72it/s]


🔁 Epoch 1 loss: 0.6925
✅ F1: 0.4350, Precision: 0.5767, Recall: 0.5147


Fold 1 Epoch 2: 100%|██████████| 147/147 [00:04<00:00, 31.59it/s]


🔁 Epoch 2 loss: 0.6896
✅ F1: 0.3943, Precision: 0.5002, Recall: 0.5001


...

🔁 Epoch 8 loss: 0.6271
✅ F1: 0.5458, Precision: 0.5580, Recall: 0.5492


Fold 5 Epoch 9: 100%|██████████| 147/147 [00:04<00:00, 31.06it/s]


🔁 Epoch 9 loss: 0.6197
✅ F1: 0.5745, Precision: 0.5743, Recall: 0.5764


Fold 5 Epoch 10: 100%|██████████| 147/147 [00:04<00:00, 31.03it/s]


🔁 Epoch 10 loss: 0.6061
✅ F1: 0.5834, Precision: 0.5879, Recall: 0.5925
📊 Matriz de confusión:
[[206 151]
 [ 89 138]]
              precision    recall  f1-score   support

        male       0.70      0.58      0.63       357
      female       0.48      0.61      0.53       227

    accuracy                           0.59       584
   macro avg       0.59      0.59      0.58       584
weighted avg       0.61      0.59      0.59       584


📊 F1 macro por fold: [0.5964, 0.6002, 0.5972, 0.6256, 0.5834]
🏁 F1 macro promedio final: 0.6006


In [23]:
import pandas as pd
import numpy as np
import random
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Semillas
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# Config
MODEL_NAME = "cardiffnlp/twitter-roberta-base"
TARGET = "gender"
CSV_PATH = "features_linguisticas_es_con_glove_con_ideology.csv"
COHEN_PATH = "cohens_results.csv"
COHEN_THRESHOLD = 0.10
BATCH_SIZE = 16
EPOCHS = 10
LR = 3e-5
FOLDS = 5
EARLY_STOPPING_PATIENCE = 4
MAX_LENGTH = 128

# Datos
df = pd.read_csv(CSV_PATH)
cohen_df = pd.read_csv(COHEN_PATH)

def normalize_gender(g):
    if pd.isnull(g): return np.nan
    g = str(g).strip().lower()
    if g in ["m", "male", "masculino"]: return "male"
    elif g in ["f", "female", "femenino"]: return "female"
    return np.nan

df["gender"] = df["gender"].apply(normalize_gender)
df = df[df["gender"].isin(["male", "female"])].copy()
df["label"] = df["gender"].map({"male": 0, "female": 1})

# Variables relevantes
selected_vars = cohen_df[
    (cohen_df["target"] == TARGET) &
    (cohen_df["cohens_d"].abs() > COHEN_THRESHOLD)
]["variable"].unique().tolist()

print(f"🔎 Usando {len(selected_vars)} variables GloVe con |d| > {COHEN_THRESHOLD}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
text_col = "clean_text"
df = df[[text_col, "label"] + selected_vars].dropna().reset_index(drop=True)

# Dataset
class MultiModalDataset(Dataset):
    def __init__(self, texts, nums, labels):
        self.texts = texts
        self.nums = nums
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        enc = tokenizer(self.texts[idx], truncation=True, padding="max_length", max_length=MAX_LENGTH, return_tensors="pt")
        item = {k: v.squeeze() for k, v in enc.items()}
        item["nums"] = torch.tensor(self.nums[idx], dtype=torch.float)
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

# Modelo
class TransformerWithTabular(nn.Module):
    def __init__(self, transformer_name, num_tabular_features):
        super().__init__()
        self.transformer = AutoModel.from_pretrained(transformer_name)
        for name, param in self.transformer.named_parameters():
            if "encoder.layer.11" not in name:
                param.requires_grad = False

        self.tabular_net = nn.Sequential(
            nn.Linear(num_tabular_features, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 64)
        )
        self.classifier = nn.Sequential(
            nn.Linear(768 + 64, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 2)
        )

    def forward(self, input_ids, attention_mask, nums):
        out = self.transformer(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = out.last_hidden_state[:, 0]
        tabular_out = self.tabular_net(nums)
        combined = torch.cat([cls_output, tabular_out], dim=1)
        return self.classifier(combined)

# Preparación
texts = df[text_col].tolist()
features = df[selected_vars].values.astype(np.float32)
labels = df["label"].values

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_weights = compute_class_weight("balanced", classes=np.unique(labels), y=labels)
weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)

skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)
f1_scores = []

# Entrenamiento
for fold, (train_idx, val_idx) in enumerate(skf.split(texts, labels)):
    print(f"\n📦 Fold {fold+1}/{FOLDS}")

    X_train_texts = [texts[i] for i in train_idx]
    X_val_texts = [texts[i] for i in val_idx]
    y_train = labels[train_idx]
    y_val = labels[val_idx]
    # Oversampling manual de la clase minoritaria (female)
    X_train_feats_orig = features[train_idx]
    X_val_feats = features[val_idx]
    
    scaler = StandardScaler()
    X_train_feats_scaled = scaler.fit_transform(X_train_feats_orig)
    X_val_feats_scaled = scaler.transform(X_val_feats)
    
    X_train_texts_aug = list(X_train_texts)
    X_train_feats_aug = list(X_train_feats_scaled)
    y_train_aug = list(y_train)
    
    # Duplicar instancias de clase 'female' (label = 1)
    for i in range(len(y_train)):
        if y_train[i] == 1:
            X_train_texts_aug.append(X_train_texts[i])
            X_train_feats_aug.append(X_train_feats_scaled[i])
            y_train_aug.append(1)
    
    # Convertir a arrays
    X_train_feats_aug = np.array(X_train_feats_aug)
    y_train_aug = np.array(y_train_aug)

    scaler = StandardScaler()
    X_train_feats = scaler.fit_transform(features[train_idx])
    X_val_feats = scaler.transform(features[val_idx])

    train_ds = MultiModalDataset(X_train_texts, X_train_feats, y_train)
    val_ds = MultiModalDataset(X_val_texts, X_val_feats, y_val)
    train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE)

    model = TransformerWithTabular(MODEL_NAME, len(selected_vars)).to(device)
    opt = torch.optim.AdamW(model.parameters(), lr=LR)
    scheduler = ReduceLROnPlateau(opt, mode='max', factor=0.5, patience=2)
    loss_fn = nn.CrossEntropyLoss(weight=weights_tensor)

    best_f1 = 0
    patience = EARLY_STOPPING_PATIENCE

    for epoch in range(EPOCHS):
        model.train()
        total_loss = 0
        for batch in tqdm(train_dl, desc=f"Fold {fold+1} Epoch {epoch+1}"):
            input_ids = batch["input_ids"].to(device)
            attn = batch["attention_mask"].to(device)
            nums = batch["nums"].to(device)
            lbls = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attn, nums=nums)
            loss = loss_fn(outputs, lbls)

            opt.zero_grad()
            loss.backward()
            opt.step()
            total_loss += loss.item()
        print(f"🔁 Epoch {epoch+1} loss: {total_loss/len(train_dl):.4f}")

        model.eval()
        all_preds, all_lbls = [], []
        with torch.no_grad():
            for batch in val_dl:
                input_ids = batch["input_ids"].to(device)
                attn = batch["attention_mask"].to(device)
                nums = batch["nums"].to(device)
                lbls = batch["labels"].to(device)

                outputs = model(input_ids=input_ids, attention_mask=attn, nums=nums)
                preds = torch.argmax(outputs, dim=1).cpu().numpy()
                all_preds.extend(preds)
                all_lbls.extend(lbls.cpu().numpy())

        f1 = f1_score(all_lbls, all_preds, average="macro")
        prec = precision_score(all_lbls, all_preds, average="macro", zero_division=0)
        rec = recall_score(all_lbls, all_preds, average="macro", zero_division=0)
        print(f"✅ F1: {f1:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}")

        scheduler.step(f1)

        if f1 > best_f1:
            best_f1 = f1
            patience = EARLY_STOPPING_PATIENCE
        else:
            patience -= 1
            if patience == 0:
                print("⏹️ Early stopping")
                break

    f1_scores.append(best_f1)
    print("📊 Matriz de confusión:")
    print(confusion_matrix(all_lbls, all_preds))
    from sklearn.metrics import classification_report
    print(classification_report(all_lbls, all_preds, target_names=["male", "female"]))

# Resultados finales
f1_avg = np.mean(f1_scores)
print("\n📊 F1 macro por fold:", [round(f, 4) for f in f1_scores])
print(f"🏁 F1 macro promedio final: {f1_avg:.4f}")


🔎 Usando 5 variables GloVe con |d| > 0.1

📦 Fold 1/5


Fold 1 Epoch 1: 100%|██████████| 147/147 [00:04<00:00, 31.24it/s]


🔁 Epoch 1 loss: 0.6933
✅ F1: 0.3790, Precision: 0.3051, Recall: 0.5000


Fold 1 Epoch 2: 100%|██████████| 147/147 [00:04<00:00, 31.45it/s]


🔁 Epoch 2 loss: 0.6926
✅ F1: 0.4595, Precision: 0.5770, Recall: 0.5505


...

🔁 Epoch 8 loss: 0.6503
✅ F1: 0.5230, Precision: 0.6086, Recall: 0.5529


Fold 5 Epoch 9: 100%|██████████| 147/147 [00:04<00:00, 31.37it/s]


🔁 Epoch 9 loss: 0.6369
✅ F1: 0.5837, Precision: 0.5940, Recall: 0.5836


Fold 5 Epoch 10: 100%|██████████| 147/147 [00:04<00:00, 31.38it/s]


🔁 Epoch 10 loss: 0.6181
✅ F1: 0.5966, Precision: 0.6002, Recall: 0.5954
📊 Matriz de confusión:
[[260  97]
 [122 105]]
              precision    recall  f1-score   support

        male       0.68      0.73      0.70       357
      female       0.52      0.46      0.49       227

    accuracy                           0.62       584
   macro avg       0.60      0.60      0.60       584
weighted avg       0.62      0.62      0.62       584


📊 F1 macro por fold: [0.5882, 0.55, 0.5957, 0.6066, 0.5966]
🏁 F1 macro promedio final: 0.5874


In [20]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight

# Configuración
MODEL_NAME = "cardiffnlp/twitter-roberta-base"
TARGET = "ideology"
COHENS_D_FILE = "cohens_results.csv"
BATCH_SIZE = 16
EPOCHS = 10
LR = 1e-5
FOLDS = 5
COHEN_THRESHOLD = 0.050

# Leer CSVs
df = pd.read_csv("features_linguisticas_es_con_glove_con_ideology.csv")
cohen_df = pd.read_csv(COHENS_D_FILE)



# Filtrar solo izquierda / derecha
df = df[df["ideology"].isin(["izquierda", "derecha"])].copy()

# Crear label binario
df["label"] = df["ideology"].map({"izquierda": 0, "derecha": 1})


# Variables con d > 0.10 para ideology, excluyendo GloVe
selected_vars = cohen_df[
    (cohen_df["target"] == TARGET) & 
    (cohen_df["cohens_d"].abs() > COHEN_THRESHOLD) &
    (~cohen_df["variable"].str.startswith("XWE-"))
]["variable"].unique().tolist()

print(f"🔎 Variables seleccionadas para '{TARGET}' (|d| > {COHEN_THRESHOLD}): {selected_vars}")

# Limpiar y preparar
text_col = "clean_text"
df = df[[text_col, "label"] + selected_vars].dropna().reset_index(drop=True)

# Normalizar variables numéricas
scaler = StandardScaler()
df[selected_vars] = scaler.fit_transform(df[selected_vars])

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Dataset personalizado
class MultiModalDataset(Dataset):
    def __init__(self, texts, nums, labels):
        self.texts = texts
        self.nums = nums
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        enc = tokenizer(self.texts[idx], truncation=True, padding="max_length", max_length=128, return_tensors="pt")
        item = {k: v.squeeze() for k, v in enc.items()}
        item["nums"] = torch.tensor(self.nums[idx], dtype=torch.float)
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

# Modelo
class TransformerWithTabular(nn.Module):
    def __init__(self, transformer_name, num_tabular_features):
        super().__init__()
        self.transformer = AutoModel.from_pretrained(transformer_name)
        self.tabular_net = nn.Sequential(
            nn.Linear(num_tabular_features, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32)
        )
        self.classifier = nn.Sequential(
            nn.Linear(768 + 32, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 2)
        )

    def forward(self, input_ids, attention_mask, nums):
        out = self.transformer(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = out.last_hidden_state[:, 0]
        tabular_out = self.tabular_net(nums)
        combined = torch.cat([cls_output, tabular_out], dim=1)
        return self.classifier(combined)

# Preparar entrenamiento
texts = df[text_col].tolist()
features = df[selected_vars].values.astype(np.float32)
labels = df["label"].values

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_weights = compute_class_weight("balanced", classes=np.unique(labels), y=labels)
weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)

skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)
f1_scores = []

for fold, (train_idx, val_idx) in enumerate(skf.split(texts, labels)):
    print(f"\n📦 Fold {fold+1}/{FOLDS}")
    train_ds = MultiModalDataset([texts[i] for i in train_idx], features[train_idx], labels[train_idx])
    val_ds = MultiModalDataset([texts[i] for i in val_idx], features[val_idx], labels[val_idx])
    train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE)

    model = TransformerWithTabular(MODEL_NAME, len(selected_vars)).to(device)
    opt = torch.optim.AdamW(model.parameters(), lr=LR)
    loss_fn = nn.CrossEntropyLoss(weight=weights_tensor)

    # Entrenamiento
    model.train()
    for epoch in range(EPOCHS):
        total_loss = 0
        for batch in tqdm(train_dl, desc=f"Fold {fold+1} Epoch {epoch+1}"):
            input_ids = batch["input_ids"].to(device)
            attn = batch["attention_mask"].to(device)
            nums = batch["nums"].to(device)
            lbls = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attn, nums=nums)
            loss = loss_fn(outputs, lbls)

            opt.zero_grad()
            loss.backward()
            opt.step()
            total_loss += loss.item()
        print(f"🔁 Epoch {epoch+1} loss: {total_loss/len(train_dl):.4f}")

    # Evaluación
    model.eval()
    all_preds, all_lbls = [], []
    with torch.no_grad():
        for batch in val_dl:
            input_ids = batch["input_ids"].to(device)
            attn = batch["attention_mask"].to(device)
            nums = batch["nums"].to(device)
            lbls = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attn, nums=nums)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_lbls.extend(lbls.cpu().numpy())

    f1 = f1_score(all_lbls, all_preds, average="macro")
    f1_scores.append(f1)
    print(f"✅ F1 Fold {fold+1}: {f1:.4f}")
    from sklearn.metrics import classification_report
    print(classification_report(all_lbls, all_preds, target_names=["izquierda", "derecha"]))

# Final
f1_avg = np.mean(f1_scores)
print("\n📊 F1 macro por fold:", [round(f, 4) for f in f1_scores])
print(f"🏁 F1 macro promedio final: {f1_avg:.4f}")


🔎 Variables seleccionadas para 'ideology' (|d| > 0.05): ['Xwords', 'Xmax_length', 'Xstop', 'Xcharacter', 'Xcapital', 'Xpunctuation', 'Xword_par', 'Xchar_par', 'Xprep', 'Xadv', 'Xnouns', 'Xpronouns', 'Xconj', 'tweet_id']

📦 Fold 1/5


2025-06-12 13:08:02.523687: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-12 13:08:02.532788: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749726482.548060 1674230 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749726482.552598 1674230 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749726482.564969 1674230 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

🔁 Epoch 1 loss: 0.6918


Fold 1 Epoch 2: 100%|██████████| 147/147 [00:13<00:00, 10.65it/s]


🔁 Epoch 2 loss: 0.6724


...

🔁 Epoch 8 loss: 0.1971


Fold 5 Epoch 9: 100%|██████████| 147/147 [00:14<00:00, 10.40it/s]


🔁 Epoch 9 loss: 0.1114


Fold 5 Epoch 10: 100%|██████████| 147/147 [00:14<00:00, 10.40it/s]


🔁 Epoch 10 loss: 0.0953
✅ F1 Fold 5: 0.5922
              precision    recall  f1-score   support

   izquierda       0.52      0.41      0.46       222
     derecha       0.68      0.77      0.72       362

    accuracy                           0.63       584
   macro avg       0.60      0.59      0.59       584
weighted avg       0.62      0.63      0.62       584


📊 F1 macro por fold: [0.6379, 0.5857, 0.6317, 0.6766, 0.5922]
🏁 F1 macro promedio final: 0.6248


In [24]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight

# Configuración
MODEL_NAME = "bert-base-uncased"
TARGET = "ideology"
COHENS_D_FILE = "cohens_results.csv"
BATCH_SIZE = 16
EPOCHS = 10
LR = 1e-5
FOLDS = 5
COHEN_THRESHOLD = 0.050

# Leer CSVs
df = pd.read_csv("features_linguisticas_es_con_glove_con_ideology.csv")
cohen_df = pd.read_csv(COHENS_D_FILE)



# Filtrar solo izquierda / derecha
df = df[df["ideology"].isin(["izquierda", "derecha"])].copy()

# Crear label binario
df["label"] = df["ideology"].map({"izquierda": 0, "derecha": 1})


# Variables con d > 0.10 para ideology, excluyendo GloVe
selected_vars = cohen_df[
    (cohen_df["target"] == TARGET) & 
    (cohen_df["cohens_d"].abs() > COHEN_THRESHOLD) &
    (~cohen_df["variable"].str.startswith("XWE-"))
]["variable"].unique().tolist()

print(f"🔎 Variables seleccionadas para '{TARGET}' (|d| > {COHEN_THRESHOLD}): {selected_vars}")

# Limpiar y preparar
text_col = "clean_text"
df = df[[text_col, "label"] + selected_vars].dropna().reset_index(drop=True)

# Normalizar variables numéricas
scaler = StandardScaler()
df[selected_vars] = scaler.fit_transform(df[selected_vars])

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Dataset personalizado
class MultiModalDataset(Dataset):
    def __init__(self, texts, nums, labels):
        self.texts = texts
        self.nums = nums
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        enc = tokenizer(self.texts[idx], truncation=True, padding="max_length", max_length=128, return_tensors="pt")
        item = {k: v.squeeze() for k, v in enc.items()}
        item["nums"] = torch.tensor(self.nums[idx], dtype=torch.float)
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

# Modelo
class TransformerWithTabular(nn.Module):
    def __init__(self, transformer_name, num_tabular_features):
        super().__init__()
        self.transformer = AutoModel.from_pretrained(transformer_name)
        self.tabular_net = nn.Sequential(
            nn.Linear(num_tabular_features, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32)
        )
        self.classifier = nn.Sequential(
            nn.Linear(768 + 32, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 2)
        )

    def forward(self, input_ids, attention_mask, nums):
        out = self.transformer(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = out.last_hidden_state[:, 0]
        tabular_out = self.tabular_net(nums)
        combined = torch.cat([cls_output, tabular_out], dim=1)
        return self.classifier(combined)

# Preparar entrenamiento
texts = df[text_col].tolist()
features = df[selected_vars].values.astype(np.float32)
labels = df["label"].values

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_weights = compute_class_weight("balanced", classes=np.unique(labels), y=labels)
weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)

skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)
f1_scores = []

for fold, (train_idx, val_idx) in enumerate(skf.split(texts, labels)):
    print(f"\n📦 Fold {fold+1}/{FOLDS}")
    train_ds = MultiModalDataset([texts[i] for i in train_idx], features[train_idx], labels[train_idx])
    val_ds = MultiModalDataset([texts[i] for i in val_idx], features[val_idx], labels[val_idx])
    train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE)

    model = TransformerWithTabular(MODEL_NAME, len(selected_vars)).to(device)
    opt = torch.optim.AdamW(model.parameters(), lr=LR)
    loss_fn = nn.CrossEntropyLoss(weight=weights_tensor)

    # Entrenamiento
    model.train()
    for epoch in range(EPOCHS):
        total_loss = 0
        for batch in tqdm(train_dl, desc=f"Fold {fold+1} Epoch {epoch+1}"):
            input_ids = batch["input_ids"].to(device)
            attn = batch["attention_mask"].to(device)
            nums = batch["nums"].to(device)
            lbls = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attn, nums=nums)
            loss = loss_fn(outputs, lbls)

            opt.zero_grad()
            loss.backward()
            opt.step()
            total_loss += loss.item()
        print(f"🔁 Epoch {epoch+1} loss: {total_loss/len(train_dl):.4f}")

    # Evaluación
    model.eval()
    all_preds, all_lbls = [], []
    with torch.no_grad():
        for batch in val_dl:
            input_ids = batch["input_ids"].to(device)
            attn = batch["attention_mask"].to(device)
            nums = batch["nums"].to(device)
            lbls = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attn, nums=nums)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_lbls.extend(lbls.cpu().numpy())

    f1 = f1_score(all_lbls, all_preds, average="macro")
    f1_scores.append(f1)
    print(f"✅ F1 Fold {fold+1}: {f1:.4f}")
    from sklearn.metrics import classification_report
    print(classification_report(all_lbls, all_preds, target_names=["izquierda", "derecha"]))

# Final
f1_avg = np.mean(f1_scores)
print("\n📊 F1 macro por fold:", [round(f, 4) for f in f1_scores])
print(f"🏁 F1 macro promedio final: {f1_avg:.4f}")


🔎 Variables seleccionadas para 'ideology' (|d| > 0.05): ['Xwords', 'Xmax_length', 'Xstop', 'Xcharacter', 'Xcapital', 'Xpunctuation', 'Xword_par', 'Xchar_par', 'Xprep', 'Xadv', 'Xnouns', 'Xpronouns', 'Xconj', 'tweet_id']

📦 Fold 1/5


Fold 1 Epoch 1: 100%|██████████| 147/147 [00:13<00:00, 10.83it/s]


🔁 Epoch 1 loss: 0.6950


Fold 1 Epoch 2: 100%|██████████| 147/147 [00:13<00:00, 10.76it/s]


🔁 Epoch 2 loss: 0.6696


...

🔁 Epoch 8 loss: 0.1259


Fold 5 Epoch 9: 100%|██████████| 147/147 [00:13<00:00, 10.74it/s]


🔁 Epoch 9 loss: 0.0880


Fold 5 Epoch 10: 100%|██████████| 147/147 [00:13<00:00, 10.72it/s]


🔁 Epoch 10 loss: 0.0727
✅ F1 Fold 5: 0.5576
              precision    recall  f1-score   support

   izquierda       0.53      0.29      0.37       222
     derecha       0.66      0.85      0.74       362

    accuracy                           0.63       584
   macro avg       0.60      0.57      0.56       584
weighted avg       0.61      0.63      0.60       584


📊 F1 macro por fold: [0.6058, 0.6174, 0.5901, 0.6582, 0.5576]
🏁 F1 macro promedio final: 0.6058
